<a href="https://colab.research.google.com/github/yutao-data/Financial-Calculator/blob/main/investment_daily_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tushare
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.8.0
    Uninstalling websocket-client-1.8.0:
      Successfully uninstalled websocket-client-1.8.0
Mounted at /content/drive


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts
import os
from datetime import datetime
import time

# Initialize the Tushare API
pro = ts.pro_api('xxxxx')

drive_path = '/content/drive/My Drive/tushare_data/daily_status'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

In [27]:
today = datetime.today().strftime('%Y%m%d')

index_list = ['000001.SH', '399001.SZ']
index_data_1 = pro.index_daily(ts_code=index_list[0], start_date=today, end_date=today)
index_data_2 = pro.index_daily(ts_code=index_list[1], start_date=today, end_date=today)
index_data = pd.concat([index_data_1, index_data_2])

index_data_path = os.path.join(drive_path, f'index_data_{today}.csv')
index_data.to_csv(index_data_path, index=False)

stock_data = pro.daily(trade_date=today)

stock_data_path = os.path.join(drive_path, f'stock_data_{today}.csv')
stock_data.to_csv(stock_data_path, index=False)

total_volume = stock_data['vol'].sum() / 1e8  # 转换为亿股
total_turnover = stock_data['amount'].sum() / 1e5  # 转换为亿元

rise_count = (stock_data['pct_chg'] > 0).sum()  # 上涨家数
fall_count = (stock_data['pct_chg'] < 0).sum()  # 下跌家数

moneyflow_hsgt = pro.moneyflow_hsgt(trade_date=today)
moneyflow_hsgt_path = os.path.join(drive_path, f'moneyflow_hsgt_{today}.csv')
moneyflow_hsgt.to_csv(moneyflow_hsgt_path, index=False)

northbound_inflow = moneyflow_hsgt['north_money'].sum() / 1e2  # 转换为亿元
southbound_inflow = moneyflow_hsgt['south_money'].sum() / 1e2  # 转换为亿元

daily_basic = pro.daily_basic(trade_date=today)

daily_basic_path = os.path.join(drive_path, f'daily_basic_{today}.csv')
daily_basic.to_csv(daily_basic_path, index=False)

total_market_cap = daily_basic['total_mv'].sum() / 1e4  # 转换为亿元
total_pe = daily_basic['pe'].mean()

# print(f"今日两市成交额：{total_turnover:.2f} 亿元")
# print(f"今日上涨家数：{rise_count}")
# print(f"今日下跌家数：{fall_count}")
# print(f"今日北向资金净流入：{northbound_inflow:.2f} 亿元")
# print(f"今日南向资金净流入：{southbound_inflow:.2f} 亿元")
# print(f"今日市场平均市盈率：{total_pe:.2f}")

print(f"今日两市成交额：{total_turnover:.2f} 亿元, 上涨：{rise_count}家, 待涨：{fall_count}家, 北向资金净流入：{northbound_inflow:.2f} 亿元, 内资净流入：{southbound_inflow:.2f} 亿元, 市场平均市盈率：{total_pe:.2f}。")

今日两市成交额：5896.99 亿元, 上涨：2341家, 待涨：2741家, 北向资金净流入：-49.85 亿元, 内资净流入：7.45 亿元, 市场平均市盈率：58.31。
